In [1]:
import sys;
sys.path.insert(0, '..')

## Chapter 5 Code Snippets and Listings

### Phase oracles (section 5.1.1)

A *phase oracle* rotates the amplitudes of the "good" outcomes by $180^\circ$. Rotating a complex number by $180^\circ$ is the same as multiplying it by -1.

We will start with a classical implementation of phase oracles.


The good outcomes can be specified classically with a predicate that returns `True` for a good outcome and `False` otherwise. For example, if we have a problem with one good outcome, 3, we can define the predicate below:

In [2]:
predicate = lambda k: True if k == 3 else False

Say we have 8 possible outcomes ($n = 3$). We can use the `predicate` function to list the good outcomes:

In [3]:
n = 3
print(f'\nGood outcomes: {[k for k in range(2**n) if predicate(k)]}')


Good outcomes: [3]


We can use this predicate to implement an oracle that takes any state and multiplies the amplitudes of the good states by -1:

In [4]:
def classical_phase_oracle(state, predicate):
    for item in range(len(state)):
        if predicate(item):
            state[item] *= -1

Let's start with an example state with $n = 3$ qubits in a uniform superposition where the magnitudes of all amplitudes are equal.

In [5]:
from math import sqrt

n = 3
state = [1/sqrt(2**n) for _ in range(2**n)]

In [6]:
from util import print_state_table

print_state_table(state)


Outcome  Binary  Amplitude           Direction  Magnitude  Amplitude Bar             Probability
------------------------------------------------------------------------------------------------
0        000     0.3536 + i0.0000       0.00°   0.3536     ████████                  0.125 
1        001     0.3536 + i0.0000       0.00°   0.3536     ████████                  0.125 
2        010     0.3536 + i0.0000       0.00°   0.3536     ████████                  0.125 
3        011     0.3536 + i0.0000       0.00°   0.3536     ████████                  0.125 
4        100     0.3536 + i0.0000       0.00°   0.3536     ████████                  0.125 
5        101     0.3536 + i0.0000       0.00°   0.3536     ████████                  0.125 
6        110     0.3536 + i0.0000       0.00°   0.3536     ████████                  0.125 
7        111     0.3536 + i0.0000       0.00°   0.3536     ████████                  0.125 



We can apply the classical phase oracle to this state with the following code:

In [7]:
classical_phase_oracle(state, predicate)

In [8]:
print('\nState after oracle is applied, changing the direction of good outcomes')
print_state_table(state)


State after oracle is applied, changing the direction of good outcomes

Outcome  Binary  Amplitude           Direction  Magnitude  Amplitude Bar             Probability
------------------------------------------------------------------------------------------------
0        000     0.3536 + i0.0000       0.00°   0.3536     ████████                  0.125 
1        001     0.3536 + i0.0000       0.00°   0.3536     ████████                  0.125 
2        010     0.3536 + i0.0000       0.00°   0.3536     ████████                  0.125 
3        011    -0.3536 + i0.0000     180.00°   0.3536     ████████                  0.125 
4        100     0.3536 + i0.0000       0.00°   0.3536     ████████                  0.125 
5        101     0.3536 + i0.0000       0.00°   0.3536     ████████                  0.125 
6        110     0.3536 + i0.0000       0.00°   0.3536     ████████                  0.125 
7        111     0.3536 + i0.0000       0.00°   0.3536     ████████                  0.12

### Bit oracles (section 5.1.2)

A *bit oracle* entangles the good outcomes with an additional qubit, which we will call the "tag bit".

The function `c_bit_oracle` is a classical implementation of a bit oracle:

In [9]:
def classical_bit_oracle(state, predicate):
    N = len(state)
    # Adding a qubit doubles the number of possible outcomes, and therefore the number of amplitudes
    state = state + [0 for _ in range(N)]
    # We find the amplitude corresponding to the outcome with 1 in the tag bit position by adding N to the amplitude index
    for item in range(N):
        if predicate(item):
            state[N + item] = state[item]
            state[item] = 0
    return state

Let's apply this oracle to a state with $n = 3$ qubits, where 3 is the good state:

In [10]:
predicate = lambda k: True if k == 3 else False

n = 3
state = [1/sqrt(2**n) for _ in range(2**n)]

tag_state = classical_bit_oracle(state, predicate)

In [11]:
print_state_table(tag_state)


Outcome  Binary  Amplitude           Direction  Magnitude  Amplitude Bar             Probability
------------------------------------------------------------------------------------------------
0        0000    0.3536 + i0.0000       0.00°   0.3536     ████████                  0.125 
1        0001    0.3536 + i0.0000       0.00°   0.3536     ████████                  0.125 
2        0010    0.3536 + i0.0000       0.00°   0.3536     ████████                  0.125 
3        0011    0.0000 + i0.0000               0                                    0     
4        0100    0.3536 + i0.0000       0.00°   0.3536     ████████                  0.125 
5        0101    0.3536 + i0.0000       0.00°   0.3536     ████████                  0.125 
6        0110    0.3536 + i0.0000       0.00°   0.3536     ████████                  0.125 
7        0111    0.3536 + i0.0000       0.00°   0.3536     ████████                  0.125 
8        1000    0.0000 + i0.0000               0                    

We can also apply this oracle to a random state generated using our `generate_state` function:

In [12]:
from util import generate_state

n = 3
state = generate_state(n, seed=777)

In [13]:
print_state_table(state)


Outcome  Binary  Amplitude           Direction  Magnitude  Amplitude Bar             Probability
------------------------------------------------------------------------------------------------
0        000    -0.1673 - i0.1616    -135.99°   0.2327     █████                     0.0541
1        001    -0.2493 + i0.2074     140.24°   0.3243     ███████                   0.1052
2        010    -0.0715 + i0.2862     104.30°   0.295      ███████                   0.087 
3        011     0.2521 - i0.0108      -2.45°   0.2524     ██████                    0.0637
4        100     0.2543 - i0.1212     -25.48°   0.2817     ██████                    0.0793
5        101    -0.1503 - i0.3937    -110.89°   0.4214     ██████████                0.1776
6        110     0.3562 - i0.3016     -40.26°   0.4667     ███████████               0.2178
7        111     0.3692 + i0.2811      37.28°   0.464      ███████████               0.2153



In [14]:
state = classical_bit_oracle(state, predicate)

In [15]:
print_state_table(state)


Outcome  Binary  Amplitude           Direction  Magnitude  Amplitude Bar             Probability
------------------------------------------------------------------------------------------------
0        0000   -0.1673 - i0.1616    -135.99°   0.2327     █████                     0.0541
1        0001   -0.2493 + i0.2074     140.24°   0.3243     ███████                   0.1052
2        0010   -0.0715 + i0.2862     104.30°   0.295      ███████                   0.087 
3        0011    0.0000 + i0.0000               0                                    0     
4        0100    0.2543 - i0.1212     -25.48°   0.2817     ██████                    0.0793
5        0101   -0.1503 - i0.3937    -110.89°   0.4214     ██████████                0.1776
6        0110    0.3562 - i0.3016     -40.26°   0.4667     ███████████               0.2178
7        0111    0.3692 + i0.2811      37.28°   0.464      ███████████               0.2153
8        1000    0.0000 + i0.0000               0                    

### Creating quantum circuits from building blocks (section 5.2.1)

**Note**: The methods `append` from listing 5.1 and `c_append` from listing 5.2 have been added to the `QuantumCircuit` class in sim_circuit.py.

For example, let's create an example three-qubit register, and a circuit with one X-gate applied to target qubit 0:

In [16]:
from sim_circuit import *

n = 3
q = QuantumRegister(n)
qc = QuantumCircuit(q)
qc.x(0)

Next, we will use the `uniform` function from chapter 4.
This function creates a circuit for encoding the uniform distribution in a state with `n` qubits.


In [17]:
def uniform(n):
    q = QuantumRegister(n)
    qc = QuantumCircuit(q)

    for i in range(len(q)):
        qc.h(q[i])

    return qc

We can apply the circuit defined in `uniform` to our three-qubit register using the `append` method:

In [18]:
n = 3
uniform_qc = uniform(n)
qc.append(uniform_qc, q) # Apply the circuit to the register q

### Phase oracle (section 5.2.2)

Listing 5.3 Function to create a phase oracle quantum circuit for a given number of qubits `n` and a set of good outcomes `items`

In [19]:
from math import pi

    
def is_bit_not_set(m, k):
    return not (m & (1 << k))

def phase_oracle_match(n, items):
    q = QuantumRegister(n)
    qc = QuantumCircuit(q)

    for m in items:
        for i in range(n):
            if is_bit_not_set(m, i):
                qc.x(q[i])

        qc.mcp(pi, [q[i] for i in range(len(q) - 1)], q[len(q) - 1])

        for i in range(n):
            if is_bit_not_set(m, i):
                qc.x(q[i])
    return qc

Let's use this function to create the oracle circuit for $n = 3$ and a single good outcome 3:

In [20]:
n = 3
items = [3]

oracle_circuit = phase_oracle_match(n, items)

In [21]:
from util_qiskit import print_circuit

print_circuit(oracle_circuit)

                      
q_0: ──────■──────────
           │          
q_1: ──────■──────────
     ┌───┐ │P(π) ┌───┐
q_2: ┤ X ├─■─────┤ X ├
     └───┘       └───┘


We can create a state in equal superposition and apply the oracle circuit defined above:

In [22]:
q = QuantumRegister(n)
qc = QuantumCircuit(q)

for i in range(n):
    qc.h(q[i])

In [23]:
qc.append(oracle_circuit, QuantumRegister(n))

In [24]:
print_state_table(qc.run())


Outcome  Binary  Amplitude           Direction  Magnitude  Amplitude Bar             Probability
------------------------------------------------------------------------------------------------
0        000     0.3536 + i0.0000       0.00°   0.3536     ████████                  0.125 
1        001     0.3536 + i0.0000       0.00°   0.3536     ████████                  0.125 
2        010     0.3536 + i0.0000       0.00°   0.3536     ████████                  0.125 
3        011    -0.3536 + i0.0000     180.00°   0.3536     ████████                  0.125 
4        100     0.3536 + i0.0000       0.00°   0.3536     ████████                  0.125 
5        101     0.3536 + i0.0000       0.00°   0.3536     ████████                  0.125 
6        110     0.3536 + i0.0000       0.00°   0.3536     ████████                  0.125 
7        111     0.3536 + i0.0000       0.00°   0.3536     ████████                  0.125 



Let's create an oracle for $n = 3$ and associated to good outcomes 1, 3, and 5:

In [25]:
n = 3
items = [1, 3, 5]

oracle_circuit = phase_oracle_match(n, items)

In [26]:
print_circuit(oracle_circuit)

                                                   
q_0: ──────■────────────────■───────────■──────────
     ┌───┐ │     ┌───┐      │     ┌───┐ │     ┌───┐
q_1: ┤ X ├─■─────┤ X ├──────■─────┤ X ├─■─────┤ X ├
     ├───┤ │P(π) ├───┤┌───┐ │P(π) ├───┤ │P(π) └───┘
q_2: ┤ X ├─■─────┤ X ├┤ X ├─■─────┤ X ├─■──────────
     └───┘       └───┘└───┘       └───┘            


Let's create a circuit which prepares the state and applies the oracle:

In [27]:
q = QuantumRegister(n)
qc = QuantumCircuit(q)

for i in range(n):
    qc.h(q[i])
    
qc.append(oracle_circuit, QuantumRegister(n))    

In [28]:
print_state_table(qc.run())


Outcome  Binary  Amplitude           Direction  Magnitude  Amplitude Bar             Probability
------------------------------------------------------------------------------------------------
0        000     0.3536 + i0.0000       0.00°   0.3536     ████████                  0.125 
1        001    -0.3536 + i0.0000     180.00°   0.3536     ████████                  0.125 
2        010     0.3536 + i0.0000       0.00°   0.3536     ████████                  0.125 
3        011    -0.3536 + i0.0000     180.00°   0.3536     ████████                  0.125 
4        100     0.3536 + i0.0000       0.00°   0.3536     ████████                  0.125 
5        101    -0.3536 + i0.0000     180.00°   0.3536     ████████                  0.125 
6        110     0.3536 + i0.0000       0.00°   0.3536     ████████                  0.125 
7        111     0.3536 + i0.0000       0.00°   0.3536     ████████                  0.125 



### Bit oracle (section 5.2.3)

Listing 5.4 Function to create a bit oracle quantum circuit for a given number of qubits `n` and a set of good outcomes `items`

In [29]:
def bit_oracle_match(n, items):
    q = QuantumRegister(n)
    a = QuantumRegister(1)
    qc = QuantumCircuit(q, a)

    for m in items:
        for i in range(n):
            if is_bit_not_set(m, i):
                qc.x(q[i])

        qc.mcx([q[i] for i in range(len(q))], a[0])

        for i in range(n):
            if is_bit_not_set(m, i):
                qc.x(q[i])
    return qc

Let's create the bit oracle circuit and apply it to our familiar example, where a state with $n = 3$ qubits is prepared using Hadamard gates and the good item is 3.

In [30]:
n = 3
items = [3]

oracle_circuit = bit_oracle_match(n, items)

q = QuantumRegister(n)
a = QuantumRegister(1)
qc = QuantumCircuit(q, a)

for i in range(n):
    qc.h(q[i])

qc.append(oracle_circuit, QuantumRegister(n + 1)) # <1>

In [31]:
print_circuit(oracle_circuit)

                     
q0_0: ───────■───────
             │       
q0_1: ───────■───────
      ┌───┐  │  ┌───┐
q0_2: ┤ X ├──■──┤ X ├
      └───┘┌─┴─┐└───┘
  q1: ─────┤ X ├─────
           └───┘     


In [32]:
print_state_table(qc.run())


Outcome  Binary  Amplitude           Direction  Magnitude  Amplitude Bar             Probability
------------------------------------------------------------------------------------------------
0        0000    0.3536 + i0.0000       0.00°   0.3536     ████████                  0.125 
1        0001    0.3536 + i0.0000       0.00°   0.3536     ████████                  0.125 
2        0010    0.3536 + i0.0000       0.00°   0.3536     ████████                  0.125 
3        0011    0.0000 + i0.0000               0.0                                  0.0   
4        0100    0.3536 + i0.0000       0.00°   0.3536     ████████                  0.125 
5        0101    0.3536 + i0.0000       0.00°   0.3536     ████████                  0.125 
6        0110    0.3536 + i0.0000       0.00°   0.3536     ████████                  0.125 
7        0111    0.3536 + i0.0000       0.00°   0.3536     ████████                  0.125 
8        1000    0.0000 + i0.0000               0.0                  

Next, let's create a bit oracle for the same state with three good outcomes:

In [33]:
n = 3
items = [1, 3, 5]

oracle_circuit = bit_oracle_match(n, items)

q = QuantumRegister(n)
a = QuantumRegister(1)
qc = QuantumCircuit(q, a)

for i in range(n):
    qc.h(q[i])

qc.append(oracle_circuit, QuantumRegister(n+1))

In [34]:
print_circuit(oracle_circuit)

                                              
q2_0: ───────■──────────────■─────────■───────
      ┌───┐  │  ┌───┐       │  ┌───┐  │  ┌───┐
q2_1: ┤ X ├──■──┤ X ├───────■──┤ X ├──■──┤ X ├
      ├───┤  │  ├───┤┌───┐  │  ├───┤  │  └───┘
q2_2: ┤ X ├──■──┤ X ├┤ X ├──■──┤ X ├──■───────
      └───┘┌─┴─┐└───┘└───┘┌─┴─┐└───┘┌─┴─┐     
  q3: ─────┤ X ├──────────┤ X ├─────┤ X ├─────
           └───┘          └───┘     └───┘     


In [35]:
print_state_table(qc.run())


Outcome  Binary  Amplitude           Direction  Magnitude  Amplitude Bar             Probability
------------------------------------------------------------------------------------------------
0        0000    0.3536 + i0.0000       0.00°   0.3536     ████████                  0.125 
1        0001    0.0000 + i0.0000               0.0                                  0.0   
2        0010    0.3536 + i0.0000       0.00°   0.3536     ████████                  0.125 
3        0011    0.0000 + i0.0000               0.0                                  0.0   
4        0100    0.3536 + i0.0000       0.00°   0.3536     ████████                  0.125 
5        0101    0.0000 + i0.0000               0.0                                  0.0   
6        0110    0.3536 + i0.0000       0.00°   0.3536     ████████                  0.125 
7        0111    0.3536 + i0.0000       0.00°   0.3536     ████████                  0.125 
8        1000    0.0000 + i0.0000               0.0                  

### From a phase oracle to a bit oracle (5.3.1)

If we have a phase oracle circuit defined using the `QuantumCircuit` class in our simulator, we can use it to create a bit oracle circuit using the function below:

In [36]:
def phase_to_bit_oracle(oracle_circuit):
    n = sum(oracle_circuit.regs) # Get the number of qubits used for the phase oracle
    q = QuantumRegister(n)
    a = QuantumRegister(1)
    qc = QuantumCircuit(q, a)
    qc.h(a[0])
    qc.c_append(oracle_circuit, a[0], q) # Apply the phase oracle circuit controlled on the ancilla qubit
    qc.h(a[0])

    return qc

For example, let's create the phase oracle circuit for $n = 3$ qubits and good outcomes 1, 3, and 5:

In [37]:
n = 3
items = [1, 3, 5]
oracle_circuit = phase_oracle_match(n, items)

In [38]:
print_circuit(oracle_circuit)

                                                   
q_0: ──────■────────────────■───────────■──────────
     ┌───┐ │     ┌───┐      │     ┌───┐ │     ┌───┐
q_1: ┤ X ├─■─────┤ X ├──────■─────┤ X ├─■─────┤ X ├
     ├───┤ │P(π) ├───┤┌───┐ │P(π) ├───┤ │P(π) └───┘
q_2: ┤ X ├─■─────┤ X ├┤ X ├─■─────┤ X ├─■──────────
     └───┘       └───┘└───┘       └───┘            


Let's use the `generate_state` function to generate a random state with $n = 3$ qubits and an ancilla qubit:

In [39]:
state = generate_state(n, seed=777) + [0 for _ in range(2**n)]

In [40]:
print_state_table(state)


Outcome  Binary  Amplitude           Direction  Magnitude  Amplitude Bar             Probability
------------------------------------------------------------------------------------------------
0        0000   -0.1673 - i0.1616    -135.99°   0.2327     █████                     0.0541
1        0001   -0.2493 + i0.2074     140.24°   0.3243     ███████                   0.1052
2        0010   -0.0715 + i0.2862     104.30°   0.295      ███████                   0.087 
3        0011    0.2521 - i0.0108      -2.45°   0.2524     ██████                    0.0637
4        0100    0.2543 - i0.1212     -25.48°   0.2817     ██████                    0.0793
5        0101   -0.1503 - i0.3937    -110.89°   0.4214     ██████████                0.1776
6        0110    0.3562 - i0.3016     -40.26°   0.4667     ███████████               0.2178
7        0111    0.3692 + i0.2811      37.28°   0.464      ███████████               0.2153
8        1000    0.0000 + i0.0000               0                    

**Note:** In this chapter, we add the method `initialize` to the `QuantumCircuit` class in sim_circuit.py, which allows us to write the state in a `QuantumCircuit` class instance.

In [41]:
q = QuantumRegister(n)
a = QuantumRegister(1)

qc = QuantumCircuit(q, a)
qc.initialize(state.copy())

qc.append(phase_to_bit_oracle(oracle_circuit), QuantumRegister(n+1))

In [42]:
print_state_table(qc.run())


Outcome  Binary  Amplitude           Direction  Magnitude  Amplitude Bar             Probability
------------------------------------------------------------------------------------------------
0        0000   -0.1673 - i0.1616    -135.99°   0.2327     █████                     0.0541
1        0001    0.0000 + i0.0000               0.0                                  0.0   
2        0010   -0.0715 + i0.2862     104.30°   0.295      ███████                   0.087 
3        0011    0.0000 + i0.0000               0.0                                  0.0   
4        0100    0.2543 - i0.1212     -25.48°   0.2817     ██████                    0.0793
5        0101    0.0000 + i0.0000               0.0                                  0.0   
6        0110    0.3562 - i0.3016     -40.26°   0.4667     ███████████               0.2178
7        0111    0.3692 + i0.2811      37.28°   0.464      ███████████               0.2153
8        1000    0.0000 + i0.0000               0.0                  

### From a bit oracle to a phase oracle (section 5.3.2)

We can use the following function to create a circuit which will act as a phase oracle, where the parameter `oracle_circuit` is a bit oracle:

In [43]:
def bit_to_phase_oracle(oracle_circuit):
    n = sum(oracle_circuit.regs)
    q = QuantumRegister(n)
    qc = QuantumCircuit(q)
    qc.append(oracle_circuit, q)
    qc.p(pi, q[len(q)-1])
    qc.append(oracle_circuit, q)

    return qc

Let's create the bit oracle circuit for our example problem where $n = 3$ qubits and good outcomes are 1, 3, and 5:

In [44]:
n = 3
items = [1, 3, 5]
oracle_circuit = bit_oracle_match(n, items)

In [45]:
print_circuit(oracle_circuit)

                                              
q4_0: ───────■──────────────■─────────■───────
      ┌───┐  │  ┌───┐       │  ┌───┐  │  ┌───┐
q4_1: ┤ X ├──■──┤ X ├───────■──┤ X ├──■──┤ X ├
      ├───┤  │  ├───┤┌───┐  │  ├───┤  │  └───┘
q4_2: ┤ X ├──■──┤ X ├┤ X ├──■──┤ X ├──■───────
      └───┘┌─┴─┐└───┘└───┘┌─┴─┐└───┘┌─┴─┐     
  q5: ─────┤ X ├──────────┤ X ├─────┤ X ├─────
           └───┘          └───┘     └───┘     


Let's generate a random state and use the bit oracle above to create a phase oracle:

In [46]:
n = 3
items = [1, 3, 5]
oracle_circuit = bit_oracle_match(n, items)

state = generate_state(n, seed=777) + [0 for _ in range(2**n)]

q = QuantumRegister(n)
a = QuantumRegister(1)

qc = QuantumCircuit(q, a)
qc.initialize(state.copy())

qc.append(bit_to_phase_oracle(oracle_circuit), QuantumRegister(n+1))

In [47]:
print_state_table(qc.run())


Outcome  Binary  Amplitude           Direction  Magnitude  Amplitude Bar             Probability
------------------------------------------------------------------------------------------------
0        0000   -0.1673 - i0.1616    -135.99°   0.2327     █████                     0.0541
1        0001    0.2493 - i0.2074     -39.76°   0.3243     ███████                   0.1052
2        0010   -0.0715 + i0.2862     104.30°   0.295      ███████                   0.087 
3        0011   -0.2521 + i0.0108     177.55°   0.2524     ██████                    0.0637
4        0100    0.2543 - i0.1212     -25.48°   0.2817     ██████                    0.0793
5        0101    0.1503 + i0.3937      69.11°   0.4214     ██████████                0.1776
6        0110    0.3562 - i0.3016     -40.26°   0.4667     ███████████               0.2178
7        0111    0.3692 + i0.2811      37.28°   0.464      ███████████               0.2153
8        1000    0.0000 + i0.0000               0.0                  

### Fibonacci numbers and the golden ratio with good outcomes (section 5.4)

We can compute the $n^{th}$ number in the Fibonacci sequence (denoted by $F_n$) using the recursive Python function below:

In [48]:
def recursive_fib(n):
    if n <= 1:
        return n
    else:
        return recursive_fib(n - 1) + recursive_fib(n - 2)

We can use the recursive function to create a list of the first 10 Fibonacci numbers:

In [49]:
[recursive_fib(n) for n in range(10)]

[0, 1, 1, 2, 3, 5, 8, 13, 21, 34]

We can create a circuit that identifies the good outcomes and makes the bad outcomes impossible. The function `fib_circuit` creates this circuit for a given number of qubits $n > 0$:

In [50]:
from math import asin

def fib_circuit(n):
    theta = 2*asin((sqrt(5) - 1)/2)

    q = QuantumRegister(n)
    qc = QuantumCircuit(q)

    for i in range(n):
        qc.ry(theta, q[i])

    for i in range(n - 1):
        qc.cry(-theta, q[i], q[i + 1])

    return qc

Let's create the circuit for one qubit:

In [51]:
qc = fib_circuit(1)

In [52]:
print_state_table(qc.run())


Outcome  Binary  Amplitude           Direction  Magnitude  Amplitude Bar             Probability
------------------------------------------------------------------------------------------------
0        0       0.7862 + i0.0000       0.00°   0.7862     ██████████████████        0.618 
1        1       0.6180 + i0.0000       0.00°   0.618      ██████████████            0.382 



For a given number of qubits $n$ we can see that:

* There are $F_{n+1}$ good outcomes with the first binary digit 0 (top half of the state table), and the amplitudes corresponding to these outcomes are all equal.
* There are $F_n$ good outcomes with the first binary digit 1 (bottom half of the state table), and the amplitudes corresponding to these outcomes are all equal.

We can check that the ratio of the probability of a good outcome that starts with 0 and that of a good outcome that starts with 1 is the golden ratio:

In [53]:
from util import is_close

qc = fib_circuit(2)
state = qc.run()

assert is_close(abs(state[0])**2/abs(state[2])**2, (1+sqrt(5))/2) # <1>
assert is_close(abs(state[1])**2/abs(state[2])**2, (1+sqrt(5))/2) # <2>

In [54]:
print_state_table(state)


Outcome  Binary  Amplitude           Direction  Magnitude  Amplitude Bar             Probability
------------------------------------------------------------------------------------------------
0        00      0.6180 + i0.0000       0.00°   0.618      ██████████████            0.382 
1        01      0.6180 + i0.0000       0.00°   0.618      ██████████████            0.382 
2        10      0.4859 + i0.0000       0.00°   0.4859     ███████████               0.2361
3        11      0.0000 + i0.0000               0.0                                  0.0   



3 qubit example:

In [55]:
qc = fib_circuit(3)
print_state_table(qc.run())


Outcome  Binary  Amplitude           Direction  Magnitude  Amplitude Bar             Probability
------------------------------------------------------------------------------------------------
0        000     0.4859 + i0.0000       0.00°   0.4859     ███████████               0.2361
1        001     0.4859 + i0.0000       0.00°   0.4859     ███████████               0.2361
2        010     0.4859 + i0.0000       0.00°   0.4859     ███████████               0.2361
3        011     0.0000 + i0.0000               0.0                                  0.0   
4        100     0.3820 + i0.0000       0.00°   0.382      █████████                 0.1459
5        101     0.3820 + i0.0000       0.00°   0.382      █████████                 0.1459
6        110     0.0000 + i0.0000               0.0                                  0.0   
7        111     0.0000 + i0.0000               0.0                                  0.0   

